# Import Dependencies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data Path

In [2]:
data_path='../MATERIALS/DATA/churn_train.csv'
df       =pd.read_csv(data_path)

Dataset information available [here](../MATERIALS/MakeIT_Desafio_Churn.pdf).

# Data Processing

### One-Hot Encoding

In [3]:
# No missing value imputation needed

# One hot encoding of 'CATEGORIA' variable.
one_hot=pd.get_dummies(df['DESCRICAO CATEGORIA'], prefix='CAT')
df     =df.join(one_hot) # join requires same index on both Dataframes

# Drop redundant variables
df.drop(columns=['CATEGORIA', 'DESCRICAO CATEGORIA'], inplace=True, axis=1)

df.head()

,ID,target,MONTANTE,DATA,CAT_Animais de estimação,CAT_Cabeleireiro e Barbeiro,CAT_Cafés e Pastelarias,CAT_Carro e Transportes,CAT_Casa,CAT_Combustível,...,CAT_Restaurantes e Bares,CAT_Saúde e Bem estar,CAT_Seguros,CAT_Subscrições,CAT_Supermercado,CAT_Taxas e Impostos,CAT_Tecnologia,CAT_Telecomunicações,CAT_Vestuário e Calçado,"CAT_Água, luz e gás"
0,991,0,-0.34,2022-04-25,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,991,0,-0.01,2022-04-25,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,991,0,-1.42,2022-04-25,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,991,0,-0.06,2022-04-25,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,991,0,-0.51,2022-04-25,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [4]:
# Separate variable types

df_real       =df[['ID','MONTANTE']] # real valued variables
df_categorical=df[[i for i in df if i not in ['MONTANTE', 'target', 'DATA']]]

## Dataset 1 
Add Features:

- **credit_count**     : Count occurances of positive **MONTANTE**;
- **debit_count**      : Count occurances of negative **MONTANTE**;
- **debit_amount**     : Sum of absolute **MONANTE** when smaller than 0;
- **credit_amount**    : Sum of **MONANTE** when greater than 0;
- **amount_moved**     : Sum of absolute **MONTANTE** values;

In [5]:
df['credit_count']     = np.where(df['MONTANTE']>0, 1, 0)
df['debit_count']      = np.where(df['MONTANTE']<0, 1, 0)
df['debit_amount']     = np.where(df['MONTANTE']>0, np.abs(df_real['MONTANTE']), 0)
df['credit_amount']    = np.where(df['MONTANTE']<0, np.abs(df_real['MONTANTE']), 0)
df['amount_moved']     = np.abs(df['MONTANTE'])

In [6]:
dataset1=df.copy()

In [7]:
dataset1.to_csv('../MATERIALS/DATA/processed_data/transaction_data.csv')

## Dataset 2 
(Compress Clients) - DISREGARD TIME DATA

- **transaction_count**: Count of client transactions;
- **credit_count**     : Count occurances of positive **MONTANTE**;
- **debit_count**      : Count occurances of negative **MONTANTE**;
- **net_count**        : linear combination **credit_count** - **debit_count**;
- **debit_amount**     : Sum of absolute **MONANTE** when smaller than 0;
- **credit_amount**    : Sum of **MONANTE** when greater than 0;
- **amount_moved**     : Sum of absolute **MONTANTE** values;

In [8]:
# CATEGORICAL

# Compress data for each client
df_occurances   = df_categorical.groupby(['ID']).sum()   #sums occurances of particular transaction
df_target       = df[['target', 'ID']].groupby('ID')['target'].agg(pd.Series.mode) # target for each client ID

# REAL

# gt -> greater than, lt -> lesser than
g = df_real.groupby('ID')['MONTANTE']
counts = g.agg(
    transaction_count =lambda s: s.count(),#count of total transactions
    credit_count =lambda s: s.gt(0).sum(),
    debit_count  =lambda s: s.lt(0).sum(),
    net_count    =lambda s: s.gt(0).sum() - s.lt(0).sum().astype(int),
    debit_amount =lambda s: s[s<0].abs().sum(),
    credit_amount=lambda s: s[s>0].sum(),
    amount_moved =lambda s: s.abs().sum())

In [9]:
dataset2=counts.join([df_occurances, df_target])

In [10]:
dataset2.to_csv('../MATERIALS/DATA/processed_data/processed_dataset2.csv')
dataset2

,transaction_count,credit_count,debit_count,net_count,debit_amount,credit_amount,amount_moved,CAT_Animais de estimação,CAT_Cabeleireiro e Barbeiro,CAT_Cafés e Pastelarias,...,CAT_Saúde e Bem estar,CAT_Seguros,CAT_Subscrições,CAT_Supermercado,CAT_Taxas e Impostos,CAT_Tecnologia,CAT_Telecomunicações,CAT_Vestuário e Calçado,"CAT_Água, luz e gás",target
ID,,,,,,,,,,,,,,,,,,,,,
1,197,20,177,-157,9134.60,12415.69,21550.29,0,0,4,...,19,8,0,24,12,4,4,10,7,0
2,188,18,170,-152,7644.70,4649.45,12294.15,0,0,0,...,14,6,0,8,38,0,4,0,9,1
3,317,75,242,-167,38217.45,41217.53,79434.98,0,0,0,...,21,5,0,6,12,0,4,3,6,0
4,384,27,357,-330,19961.62,72310.61,92272.23,0,1,0,...,6,14,0,41,39,0,11,3,8,0
5,592,219,373,-154,1245577.59,1285907.14,2531484.73,0,0,0,...,12,0,0,30,50,1,13,6,60,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1748,137,8,129,-121,3754.13,3452.45,7206.58,0,2,0,...,7,0,0,9,8,0,16,2,0,0
1749,41,6,35,-29,6762.48,1725.87,8488.35,0,0,0,...,1,0,0,3,8,0,1,0,0,1
1750,13,7,6,1,5200.54,5236.92,10437.46,0,0,0,...,0,0,0,0,1,0,0,0,0,0
